In [21]:
import os
os.chdir('/Users/mumtaz/Documents/projects/zavmo/zavmo-api/zavmo')

In [ ]:
from zavmo.helpers.chat import get_batch_openai_embedding

In [22]:
import pandas as pd
import numpy as np
from IPython.display import Markdown
from sklearn.metrics.pairwise import cosine_similarity

from dotenv import load_dotenv
load_dotenv()

True

In [23]:
import warnings
warnings.filterwarnings("ignore")

In [24]:
ofqual_SSAs = [
    "Medicine and dentistry",
    "Nursing, and subjects and vocations allied to medicine",
    "Health and social care",
    "Public services",
    "Child development and well being",
    "Science",
    "Mathematics and statistics",
    "Agriculture",
    "Horticulture and forestry",
    "Animal care and veterinary science",
    "Environmental conservation",
    "Engineering",
    "Manufacturing technologies",
    "Transportation operations and maintenance",
    "Architecture",
    "Building and construction",
    "Urban, rural and regional planning",
    "Digital technology (practitioners)",
    "Digital technology (users)",
    "Retailing and wholesaling",
    "Warehousing and distribution",
    "Service enterprises",
    "Hospitality and catering",
    "Sport, leisure and recreation",
    "Travel and tourism",
    "Performing arts",
    "Crafts, creative arts and design",
    "Media and communication",
    "Publishing and information services",
    "History",
    "Archaeology and archaeological sciences",
    "Philosophy",
    "Theology and religious studies",
    "Geography",
    "Sociology and social policy",
    "Politics",
    "Economics",
    "Anthropology",
    "Languages, literature and culture of the British isles",
    "Other languages, literature and culture",
    "Linguistics",
    "Teaching and lecturing",
    "Direct learning support",
    "Foundations for learning and life",
    "Preparation for work",
    "Accounting and finance",
    "Administration",
    "Business management",
    "Marketing and sales",
    "Law and legal services"
]

In [25]:
nos = pd.read_excel("/Users/mumtaz/Documents/projects/zavmo/complete_nos.xlsx").drop_duplicates(subset=['urn'], keep='first')
nos = nos.dropna(subset=['overview','performance_criteria', 'knowledge_and_understanding'])
nos.head(3)

,urn,suites,occupations,soc,developed_by_1,approved_on,web_link,pdf_link,overview,performance_criteria,knowledge_and_understanding,additional_information,developed_by_2,key_words,nos_title
3,PROSM09,Sawmilling,Paper and Wood Machine Operatives,8131,NSAFD v2,01/04/2020 00:00,NaN,https://files.ukstandards.org.uk/pdfs/PROSM09.pdf,This standard covers the skills and knowledge ...,You must be able to: 1. carry out risk assessm...,You need to know and 1. relevant health and sa...,NaN,NSAFD Version Number 2 Date Approved March 201...,Timber; wood; abrasive; wheel; saw; cut; sawmi...,Abrasive wheel operations in the wood industry
4,PPLRETRS17,Rail Engineering,Transport Drivers and Operatives,8153,Enginuity,15/03/2015 00:00,NaN,https://files.ukstandards.org.uk/pdfs/PPLRETRS...,This unit is about accepting and returning res...,You must be able to: P1 work safely at all tim...,You need to know and K1 the relevant health an...,"Glossary Health and safety legislation, regula...",People 1st Version number 1 Date approved Apri...,Responsibility; control; handover PPLRETRS17 A...,"Accept, and return, responsibility for the con..."
5,COGPOLY9,Polymer Processing and Related Operations,"Engineering Professionals, Manufacturing Techn...",8113,Cogent,31/01/2012 00:00,NaN,https://files.ukstandards.org.uk/pdfs/COGPOLY9...,This unit is for those with responsibility for...,You must be able to: P1 work safely at all tim...,You need to know and K1 how to access and inte...,Glossary Key terms Examples Regulations and gu...,Cogent Version number 1 Date approved January ...,"accept, verify, store, materials, process, ope...","Accept, verify and store materials required fo..."


In [27]:
len(nos['suites'].unique())

1517

In [165]:
sample_data = [f"Industry: {i['suites']}\n\nOverview: {i['overview']}" for i in nos.head(5000).to_dict(orient='records')] 

In [166]:
Markdown(sample_data[0])

Industry: Sawmilling

Overview: This standard covers the skills and knowledge required for working with abrasive wheels in the wood industry. It covers carrying out checks and adjustments, handling, storing and transporting abrasive wheels and includes dressing and mounting of abrasive wheels. This standard is suitable for operators working in the wood and timber industry. PROSM09 Abrasive wheel operations in the wood industry 1 PROSM09 Abrasive wheel operations in the wood industry

In [167]:
print(f"Sample size: {len(sample_data)}")

Sample size: 5000


In [168]:
print(f"Ofqual Sector Subject Areas: {len(ofqual_SSAs)}")

Ofqual Sector Subject Areas: 50


In [170]:
sample_embeddings = get_batch_openai_embedding(sample_data, model="text-embedding-3-large", dimensions=768)

In [171]:
category_embeddings = get_batch_openai_embedding(ofqual_SSAs, model="text-embedding-3-large", dimensions=768)

In [172]:
df = pd.DataFrame({"text": sample_data,"embedding": sample_embeddings})

In [189]:
similarities = cosine_similarity(df["embedding"].to_list(), category_embeddings)

In [197]:
df['label']    = [ofqual_SSAs[np.argmax(sim)] for sim in similarities]
df["scores"]   = [sim for sim in similarities]
df['nos_id']   = nos['urn'].tolist()[:len(sample_data)]

In [198]:
df

,text,embedding,predicted_label,scores,nos_id,label
0,Industry: Sawmilling\n\nOverview: This standar...,"[0.026389580219984055, 0.05260821804404259, -0...",Horticulture and forestry,"[0.20907517015959068, 0.23072707402558085, 0.2...",PROSM09,Horticulture and forestry
1,Industry: Rail Engineering\n\nOverview: This u...,"[-0.05731497332453728, 0.0581032857298851, -0....",Transportation operations and maintenance,"[0.15666039085735797, 0.229616114227125, 0.344...",PPLRETRS17,Transportation operations and maintenance
2,Industry: Polymer Processing and Related Opera...,"[0.004590315278619528, 0.036927931010723114, -...",Warehousing and distribution,"[0.19656724188905045, 0.23809156516698782, 0.2...",COGPOLY9,Warehousing and distribution
3,"Industry: Agricultural Crop Production, Animal...","[-0.021533822640776634, 0.052498698234558105, ...",Business management,"[0.19177822088580235, 0.24588426537641625, 0.3...",CFABAA624L,Business management
4,Industry: Offshore S1971 Elected Safety Repres...,"[0.02236284129321575, 0.001131381606683135, -0...",Health and social care,"[0.19424732792861515, 0.2270977410914082, 0.33...",COGOESR1,Health and social care
...,...,...,...,...,...,...
4995,"Industry: Dental Technology\n\nOverview: ""This...","[-0.008561796508729458, 0.032812248915433884, ...",Manufacturing technologies,"[0.36415286811855063, 0.30167710654107305, 0.2...",SFHOH30,Manufacturing technologies
4996,Industry: Dental Technology\n\nOverview: This ...,"[-0.009331854991614819, 0.03587661683559418, -...",Manufacturing technologies,"[0.37189720845176794, 0.3101215755190612, 0.30...",SFHOH30L,Manufacturing technologies
4997,Industry: Rehabilitation Technical Services\n\...,"[-0.0011902742553502321, 0.045711223036050797,...",Manufacturing technologies,"[0.30978133038333616, 0.32726042622936935, 0.3...",SFHRT6,Manufacturing technologies
4998,Industry: Dental Technology\n\nOverview: This ...,"[-0.0065032984130084515, 0.033594317734241486,...",Medicine and dentistry,"[0.4011101449699853, 0.3361162429605878, 0.358...",SFHDT04L,Medicine and dentistry


## Manual Check To Verfy If Classification Is Correct

In [199]:
checks=df.to_dict(orient='records')

In [200]:
n=3
print(f"Predicted Label: {checks[n]['label']}\n\nText")
Markdown(f"{checks[n]['text']}")

Predicted Label: Business management

Text


Industry: Agricultural Crop Production, Animal Technology, Business and Administration (2013), Equine, Horticulture, Livestock Production

Overview: This standard is about accepting instructions to take responsibility for own work and its delivery, improving own performance and behaving in a way that encourages effective working. It includes showing a willingness to take on new challenges and using feedback to agree ways in which own work might be improved. It is for administrators who accept instructions to manage and improve their own performance. Links: All categories Specific skills: 1. analysing 2. communicating 3. decision making 4. organising 5. planning 6. presenting information 7. problem solving 8. researching 9. using technology CFABAA624 Accept instructions to manage and improve own performance in a business environment 1 CFABAA624 Accept instructions to manage and improve own performance in a business environment

In [201]:
print(f"Number of unique ofqual SSAs mapped in training data: {len(df['label'].unique())}")

Number of unique ofqual SSAs mapped in training data: 41


## Missing OFFQUL SSAs

In [202]:
[s for s in ofqual_SSAs if s not in df['label'].unique()]

['Agriculture',
 'History',
 'Philosophy',
 'Geography',
 'Politics',
 'Anthropology',
 'Languages, literature and culture of the British isles',
 'Other languages, literature and culture',
 'Linguistics']

In [203]:
df.to_excel(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/classification/classification_training_data.xlsx", 
            index=False, columns=['nos_id', 'text', 'label'])